## Objective:

### 1. Extract All Play-by-Play Data

- Identify how to pull play-by-play data for all events on a given day
- Identify how to pull (or feed) all valid dates
- Iterate through valid dates and return play-by-play resultset


#### Play-by-Play Data

- `statcast_single_game` [module](https://github.com/jldbc/pybaseball/blob/master/docs/statcast_single_game.md) retrieves all statcast records from a given `game_pk`
- `statcast` (module)[https://github.com/jldbc/pybaseball/blob/master/docs/statcast.md retrieves all statcast data for a given ***date range***, with inputs:
> `statcast(start_dt=[yesterday's date], end_dt=None, team=None, verbose=True, parallel=True)`

Open question as to retrieval of dim tables, but as a raw pitch-level should suffice

> [!WARNING]
> Statcast data only goes back as far as 2008
> Will need an alternative solution for all-time


In [11]:
from pybaseball import statcast
from datetime import date, timedelta

curr_date = date.today()
bgn_date = curr_date - timedelta(days=3)

date_fmt = '%Y-%m-%d' 


data = statcast(
    start_dt=bgn_date.strftime(date_fmt),
    end_dt=curr_date.strftime(date_fmt)
)

data.head()

This is a large query, it may take a moment to complete


 25%|██▌       | 1/4 [00:00<00:00,  5.30it/s]/Users/ericclaflin/dev/curves/venv/lib/python3.9/site-packages/pybaseball/datahelpers/postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  data_copy[column] = data_copy[column].apply(pd.to_datetime, errors='ignore', format=date_format)
/Users/ericclaflin/dev/curves/venv/lib/python3.9/site-packages/pybaseball/datahelpers/postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  data_copy[column] = data_copy[column].apply(pd.to_datetime, errors='ignore', format=date_format)
/Users/ericclaflin/dev/curves/venv/lib/python3.9/site-packages/pybaseball/datahelpers/postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without pass

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,bat_speed,swing_length
3577,FF,2024-06-23,96.5,3.98,5.7,"Diekman, Jake",621550,518617,strikeout,called_strike,...,2,2,5,Standard,Standard,114,-0.014,-0.139,<NA>,<NA>
3691,FF,2024-06-23,95.6,4.11,5.72,"Diekman, Jake",621550,518617,NaN,swinging_strike,...,2,2,5,Standard,Standard,112,0.0,-0.046,41.104372,6.27007
3780,FF,2024-06-23,94.8,3.87,5.84,"Diekman, Jake",621550,518617,NaN,called_strike,...,2,2,5,Strategic,Standard,119,0.0,-0.041,<NA>,<NA>
2578,FC,2024-06-23,86.4,-2.07,5.74,"Smith, Drew",621020,622098,single,hit_into_play,...,2,2,5,Standard,Standard,149,0.009,0.159,69.84708,6.69749
2638,FF,2024-06-23,94.6,-2.11,5.64,"Smith, Drew",621020,622098,NaN,foul,...,2,2,5,Standard,Standard,214,0.0,0.0,69.515685,6.83738


In [8]:
# Confirm rough size of a single day (in season) - may help answer how we want to treat extract batch size
data.shape

(12077, 94)

121k rows over a weekend in-season non-holiday should tell us that a single day is roughly 40k records

##### Approach

Create a pydantic class for statcast_day object, including:
- An init method that creates the object and retrievers the resultset
- pydantic data validation/construction


First create a function that retrieves a dataframe for a given day

> [!NOTE]
> Refer to this [gist](https://gist.github.com/eclaflin/6857c074f949833d87aac1390f637cfc) to come back to adding logging

In [20]:

from pydantic import BaseModel, field_validator
from datetime import datetime
from typing import Any
import pandas as pd

class DateValidator(BaseModel):
    query_date: str

    @field_validator('query_date')
    @classmethod
    def validate_date_format(cls, value):
        try:
            datetime.strptime(value, '%Y-%m-%d')
        except ValueError:
            raise ValueError(f'{value} is not a valid date in the format YYYY-MM-DD')
        return value

def get_statcast_pitch_day(query_date: str) -> pd.DataFrame:
    # Validate the date string
    DateValidator(query_date=query_date)
    
    df_statcast_pitch_day = statcast(
        start_dt=query_date
    )
    
    return df_statcast_pitch_day


data = get_statcast_pitch_day('2024-06-21')

This is a large query, it may take a moment to complete


  0%|          | 0/1 [00:00<?, ?it/s]/Users/ericclaflin/dev/curves/venv/lib/python3.9/site-packages/pybaseball/datahelpers/postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  data_copy[column] = data_copy[column].apply(pd.to_datetime, errors='ignore', format=date_format)
100%|██████████| 1/1 [00:01<00:00,  1.32s/it]


Second

Create pydantic class to represent the dataframe

In [27]:
types = pd.DataFrame(data.dtypes )

types

,0
pitch_type,object
game_date,datetime64[ns]
release_speed,Float64
release_pos_x,Float64
release_pos_z,Float64
...,...
spin_axis,Int64
delta_home_win_exp,Float64
delta_run_exp,Float64
bat_speed,Float64


In [28]:
from pydantic import BaseModel 
from datetime import datetime

class StatcastPitchDay(BaseModel):
    game_pk: int
    pitch_number: int
    at_bat_number: int 
    game_date: datetime
    


In [ ]:
from typing import List

def create_statcast_pitch_day(df: pd.DataFrame) -> List(StatcastPitchDay):
    pitches = []
    
    for _, row in df.iterrows():
        pitches = StatcastPitchDay(**row.to_dict())
        pitches.append(pitch_list)
    
    return pitches

pitch_list = create_statcast_pitch_day(df)

### 2. Load Play-by-Play Data Into Postgres

- Accept play-by-play data as an input
- Append records to raw table in target


### 3. Init dbt Project Targeting Postgres


In [30]:
from pybaseball import schedule_and_record

# Game-by-game results from the Yankees' 1927 season
data = schedule_and_record(2024, "BOS")

http://www.baseball-reference.com/teams/BOS/2024-schedule-scores.shtml


/Users/ericclaflin/dev/curves/venv/lib/python3.9/site-packages/pybaseball/team_results.py:75: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Attendance'].replace(r'^Unknown$', np.nan, regex=True, inplace = True) # make this a NaN so the column can benumeric


In [31]:
data

,Date,Tm,Home_Away,Opp,W/L,R,RA,Inn,W-L,Rank,GB,Win,Loss,Save,Time,D/N,Attendance,cLI,Streak,Orig. Scheduled
1,"Thursday, Mar 28",BOS,@,SEA,W,6.0,4.0,9.0,1-0,1.0,Tied,Bello,Castillo,Jansen,2:45,N,45337.0,.93,1.0,None
2,"Friday, Mar 29",BOS,@,SEA,L,0.0,1.0,9.0,1-1,3.0,1.0,Kirby,Pivetta,Muñoz,2:24,N,30013.0,.99,-1.0,None
3,"Saturday, Mar 30",BOS,@,SEA,L-wo,3.0,4.0,10.0,1-2,4.0,2.0,Saucedo,Rodríguez,None,2:46,N,32149.0,.94,-2.0,None
4,"Sunday, Mar 31",BOS,@,SEA,W,5.0,1.0,9.0,2-2,3.0,2.0,Whitlock,Miller,Slaten,2:32,D,29331.0,.89,1.0,None
5,"Monday, Apr 1",BOS,@,OAK,W,9.0,0.0,9.0,3-2,3.0,2.0,Houck,Boyle,Anderson,2:31,N,6618.0,.98,2.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,"Tuesday, Sep 24",BOS,@,TOR,None,NaN,NaN,NaN,None,NaN,None,None,None,None,None,None,NaN,None,NaN,None
158,"Wednesday, Sep 25",BOS,@,TOR,None,NaN,NaN,NaN,None,NaN,None,None,None,None,None,None,NaN,None,NaN,None
159,"Friday, Sep 27",BOS,Home,TBR,None,NaN,NaN,NaN,None,NaN,None,None,None,None,None,None,NaN,None,NaN,None
160,"Saturday, Sep 28",BOS,Home,TBR,None,NaN,NaN,NaN,None,NaN,None,None,None,None,None,None,NaN,None,NaN,None


In [32]:
from pybaseball import team_ids

teams = team_ids(2019)


In [33]:
teams

,yearID,lgID,teamID,franchID,teamIDfg,teamIDBR,teamIDretro
2845,2019,AL,BAL,BAL,2,BAL,BAL
2846,2019,AL,BOS,BOS,3,BOS,BOS
2847,2019,AL,CHA,CHW,4,CHW,CHA
2848,2019,AL,CLE,CLE,5,CLE,CLE
2849,2019,AL,DET,DET,6,DET,DET
2850,2019,AL,HOU,HOU,21,HOU,HOU
2851,2019,AL,KCA,KCR,7,KCR,KCA
2852,2019,AL,LAA,ANA,1,LAA,ANA
2853,2019,AL,MIN,MIN,8,MIN,MIN
2854,2019,AL,NYA,NYY,9,NYY,NYA
